In [12]:
import threading
import multiprocessing
from time import sleep
import socket

from envVMWM import *
exe_location='C:\\Users\\YuHang\\Desktop\\Water_Maze\\v0.17\\VMWM.exe'
cfg_location = 'C:\\Users\\YuHang\\Desktop\\Water_Maze\\v0.17\\VMWM_data\\configuration_original.txt'

In [13]:
class Worker():

    def __init__(self,name):
        self.name = "worker_" + str(name)
        self.number = name
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1', 0))
        port = sock.getsockname()[1]
        print(sock.getsockname())
        #sock.shutdown(socket.SHUT_WR)
        self.env = VMWMGame(cfg_location,exe_location)
        self.env.reset_cfg()
        self.env.set_trial('Practice - Hills')
        self.env.set_local_host('127.0.0.1', port) # local host IP address & dynamic allocated port 
    
    def start(self):
        self.env.start()
        
    def work(self):
        self.env.start_trial()
        sleep(0.5)
        #print('OK')
        for i in range(20):
            #print('In action')
            while(not self.env.is_episode_finished()):
                #cv2.imshow('frame', env.get_screenImage())
                #cv2.waitKey(1)
                self.env.make_action(2,50)
                #print(env.get_reward())
                #print(env.get_score())
            self.env.new_episode()
            
        self.env.end_trial()
        self.env.s.close()

In [14]:
num_workers = multiprocessing.cpu_count() # Set workers ot number of available CPU threads
workers = []
# Create worker classes
for i in range(2):
    worker = Worker(i)
    workers.append(worker)
    worker.start()
    
worker_threads = []
for worker in workers:
    worker_work = lambda: worker.work()
    t = threading.Thread(target=(worker_work))
    t.start()
    sleep(1)
    worker_threads.append(t)

('127.0.0.1', 56693)
Port being used: 56693
Wait for 8s to ensure fully load up.
local host--127.0.0.1:56693
Worker #51689 set up is done.
('127.0.0.1', 56701)
Port being used: 56701
Wait for 8s to ensure fully load up.
local host--127.0.0.1:56701
Worker #51697 set up is done.
Doesn't read episode info.


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-14-50081e8e9f38>", line 11, in <lambda>
    worker_work = lambda: worker.work()
  File "<ipython-input-13-294f47b68c2d>", line 25, in work
    while(not self.env.is_episode_finished()):
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.17\Analysis\envVMWM.py", line 246, in is_episode_finished
    self.s.send(bytes("EpisodeEnd" + END_TOKEN, 'utf8'))
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine



Doesn't read episode info.


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-14-50081e8e9f38>", line 11, in <lambda>
    worker_work = lambda: worker.work()
  File "<ipython-input-13-294f47b68c2d>", line 25, in work
    while(not self.env.is_episode_finished()):
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.17\Analysis\envVMWM.py", line 246, in is_episode_finished
    self.s.send(bytes("EpisodeEnd" + END_TOKEN, 'utf8'))
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

